<a href="https://colab.research.google.com/github/mohit200522/PRODIGY_ML_TASK3/blob/main/ML_TASK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python scikit-image scikit-learn numpy tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_path = "/content/drive/MyDrive/train.zip"   # <-- Change only if needed
extract_path = "/content/dataset/"

import zipfile

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset Extracted Successfully!")


Dataset Extracted Successfully!


In [ ]:
import os
import cv2
import numpy as np
from sklearn.svm import LinearSVC
from skimage.feature import hog
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm



In [ ]:
DATASET_PATH = "/content/dataset/train/"   # Extracted folder path

images = []
labels = []

for img in tqdm(os.listdir(DATASET_PATH)):
    img_path = os.path.join(DATASET_PATH, img)

    # Label: cat = 0, dog = 1
    if "cat" in img:
        labels.append(0)
    else:
        labels.append(1)

    # Read and preprocess
    image = cv2.imread(img_path)
    image = cv2.resize(image, (128, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # HOG feature extraction
    hog_features = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )

    images.append(hog_features)

# Convert to numpy
X = np.array(images)
y = np.array(labels)

print("Feature Extraction Completed!")
print("Feature Shape:", X.shape)


100%|██████████| 25000/25000 [04:04<00:00, 102.28it/s]


Feature Extraction Completed!
Feature Shape: (25000, 8100)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 20000
Testing samples: 5000


In [ ]:
print("Training FAST Linear SVM...")
svm = LinearSVC()
svm.fit(X_train, y_train)
print("Training Completed!")


Training FAST Linear SVM...
Training Completed!


In [ ]:
y_pred = svm.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print("Model Accuracy:", acc)


Model Accuracy: 0.6848


In [ ]:
def predict_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (128, 128))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    feats = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys'
    )

    pred = svm.predict([feats])[0]
    return "DOG 🐶" if pred == 1 else "CAT 🐱"

# Test example
test_img = "/content/9435.jpg"
print(predict_image(test_img))


DOG 🐶
